앞으로 뭘하나요?  

1. PDF 파일을 다운로드 합니다.
2. Langchain의 PyPDFLoader로 PDF 파일을 읽어서 페이지 별로 저장합니다.
  - 예를 들어 27페이지의 PDF 파일이라면 27개의 문서를 얻게 됩니다.  
3. 27개의 문서들은 길이가 제각각입니다. 특정 길이(여기서는 500)로 제한하고 더 잘라줍니다.
4. 총 69개의 문서가 나오게 됩니다. 69개의 문서를 전부 임베딩(Embedding)하여 벡터 데이터베이스에 적재합니다.  
5. 벡터 데이터베이스는 Chroma 또는 Faiss를 사용할 것입니다.  
6. 질문을 입력하면 27개의 문서 중 가장 유사도가 높은 문서 4개를 출력합니다.

## 필요한 패키지 다운로드

필요한 패키지를 다운로드 합니다.  

랭체인, 벡터데이터베이스, 임베딩 모델, PDF 파일을 읽는 도구들입니다.

In [3]:
!pip install langchain chromadb transformers sentence-transformers pypdf langchain-community


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 랭체인(Langchain)을 이용한 PDF 로드 및 페이지 별 분할

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings

삼성전자 기업분석 PDF를 다운로드합니다.

In [5]:
!wget https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf

'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


PDF를 Langchain의 PyPDFLoader로 로드하고 분할합니다. 해당 PDF는 참고로 27페이지입니다.  
따라서 문서를 읽고 분할하면 총 27개의 문서가 나옵니다.

In [7]:
## pdf 파일로드 하고 쪼개기
loader = PyPDFLoader('https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf')
pages = loader.load_and_split()
print(len(pages))

Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 75 65536 (offset 0)
Ignoring wrong pointing object 85 65536 (offset 0)
Ignoring wrong pointing object 94 65536 (offset 0)
Ignoring wrong pointing object 98 65536 (offset 0)
Ignoring wrong pointing object 102 65536 (offset 0)
Ignoring wrong pointing object 109 65536 (offset 0)
Ignoring wrong pointing object 119 65536 (offset 0)
Ignoring wrong pointing object 132 65536 (offset 0)
Ignoring wrong pointing object 148 65536 (offset 0)
Ignoring wrong pointing object 176 65536 (offset 0)
Ignoring wrong pointing object 188 65536 (offset 0)
Ignoring wrong pointing object 192 65536 (offset 0)
Ignoring wrong pointing object 199 65536 (offset 0)
Ignoring wrong pointing object 203 65536 (offset 0)
Ignoring wrong pointing object 207 65536 (offset 0)
Ignoring wrong pointing object 211 65536 (offset 0)
Ignoring wrong pointing object 215 65536 (offset 0)
Ignoring wrong pointing object 219 65536 (offset 0)
Ignoring wrong poi

27


In [8]:
print(len(pages))

27


첫번째 문서를 출력해봅시다. 랭체인으로 로드한 데이터의 형식은 다음을 따릅니다.  

`Document(metadata={이 파일의 정보}, page_content='내용')`

In [9]:
pages[0].page_content

'2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등\nIM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등\nDS DRAM, NAND Flash, 모바일AP, LCD패널, OLED패널, LED 등 \n∙ 주요 사업은 전자전지기계 등 제조, 전자통신기 등 제도, 컴퓨터 등 제조, 반도체 제조·조립 등. 주요 \n제품과 구성비율은 HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등 54%, TV, 모니터, 냉장고, 세탁기, \n에어컨 등 2 4 % ,  D R A M ,  N A N D  F l a s h ,  모바일 AP 등 19% 등으로 구분 \n∙ 지역별로는 본사를 거점으로 한국 및 CE, IM 부문 산하 해외 9개 지역총괄과 DS 부문 산하 해외 5개 \n지역총괄의 생산ㆍ판매법인 등 165개의 동종업종을 영위하는 종속기업으로 구성\n∙ 국내사업장 : 수원, 구미, 기흥, 화성, 온양, 광주\n∙ 해외사업장 : 86개국 220개지점 (15년 말 기준)\n1) 연결재무재표 기준\n2) 연결재무재표 기준'

길이 500기준으로 쪼개줍니다.

In [10]:
## chunk로 쪼개기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splited_docs = text_splitter.split_documents(pages)
print(len(splited_docs))

69


이제 문서는 69개가 되었습니다.

In [11]:
splited_docs[0]

Document(metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}, page_content='2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등')

실제로 길이를 재보면 500이 넘지 않는 것을 확인할 수 있습니다.

In [12]:
len(splited_docs[0].page_content)

490

69개의 문서를 임베딩해야 합니다. 공개되어져 있는 오픈소스 임베딩을 사용해보겠습니다.  

임베딩 모델 위치: https://huggingface.co/BAAI/bge-m3

In [13]:
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

C:\Users\user\AppData\Local\Temp\ipykernel_14864\2297800112.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')


## 1. 크로마(Chroma)

다음은 Langchain에서 Chroma를 이용하여 벡터 DB 사용 시에 가장 간단한 사용법입니다.

```python
# Chroma 를 통해 벡터 저장소 생성. OpenAIEmbeddings()를 사용하거나 HuggingFaceEmbeddings()를 사용하면 된다.
chroma_db = Chroma.from_documents(splited_docs, HuggingFaceEmbeddings())

# 유사도 검색(쿼리)
similar_docs = chroma_db.similarity_search("사용자의 질문")
```

실제로 실행해봅시다.

In [14]:
## Chroma 기반 pdf(docs 벡터화)
db = Chroma.from_documents(splited_docs, model_huggingface)

print('문서의 수:', db._collection.count())

C:\ProgramData\anaconda3\Lib\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


문서의 수: 69


69개의 문서가 위에서 로드한 오픈 소스 임베딩을 바탕으로 임베딩되었다고 보면 됩니다.

In [15]:
## 질의하기
question = '삼성전자의 주요 사업영역은?'
docs = db.similarity_search(question, k=4)

상위 4개의 문서를 출력해보겠습니다.

In [16]:
len(docs)

4

In [17]:
for doc in docs:
  print(doc)
  print('--' * 100)

page_content='11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공' metadata={'creationdate': '2017-08-16T00:21:02-08:00', 'creator': 'nPDF (pdftk 1.41)', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 9, 'page_label': '10', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------

vectorize된 텍스트를 Chroma db 파일로 저장하기

In [18]:
db_toFiles = Chroma.from_documents(splited_docs, model_huggingface, persist_directory = './samsumg.db')
# db_toFiles.delete_collection()
print('문서의 수:', db_toFiles._collection.count())

문서의 수: 139


In [19]:
## 질의하기
question = '삼성전자의 주요 사업영역은?'
docs = db_toFiles.similarity_search(question, k=4)
print(len(docs))
docs[0]

4


Document(metadata={'creationdate': '2017-08-16T00:21:02-08:00', 'creator': 'nPDF (pdftk 1.41)', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 9, 'page_label': '10', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27}, page_content='11\nⅡ 기업 상세 분석\n1  사업분야(내용)\nQ1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-\ntion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문\n① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공')

In [ ]:
# 리눅스 명령어
%ls -al

total 1248
drwxr-xr-x 1 root root    4096 Feb 28 10:43 ./
drwxr-xr-x 1 root root    4096 Feb 28 10:32 ../
drwxr-xr-x 4 root root    4096 Feb 26 18:27 .config/
-rw-r--r-- 1 root root 1253369 Aug 23  2017 gsat_170823.pdf
drwxr-xr-x 2 root root    4096 Feb 28 10:35 .ipynb_checkpoints/
drwxr-xr-x 1 root root    4096 Feb 26 18:27 sample_data/
drwxr-xr-x 3 root root    4096 Feb 28 10:43 samsumg.db/


 db 파일을 읽어와서 바로 모델에 적용하여 결과를 볼 수 있습니다.

In [20]:
db_fromfile = Chroma(persist_directory='./samsumg.db', embedding_function=model_huggingface)
print('문서의 수:', db_fromfile._collection.count())

## 질의하기
question = '삼성전자의 주요 사업영역은?'
docs = db_fromfile.similarity_search(question, k=4)

print(len(docs))

문서의 수: 139
4


C:\Users\user\AppData\Local\Temp\ipykernel_14864\169561607.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db_fromfile = Chroma(persist_directory='./samsumg.db', embedding_function=model_huggingface)


In [21]:
question = '삼성전자의 주요 사업영역은?'

# TOP 3개만 추출
docs3 = db_fromfile.similarity_search_with_relevance_scores(question, k=10)

In [22]:
for doc in docs3:
  print(doc)
  print('--' * 100)

(Document(metadata={'creationdate': '2017-08-16T00:21:02-08:00', 'creator': 'nPDF (pdftk 1.41)', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 9, 'page_label': '10', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27}, page_content='11\nⅡ 기업 상세 분석\n1  사업분야(내용)\nQ1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-\ntion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문\n① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공'), 0.6830467008459198)
----------------------------------------------------------------------------------------------------------------------

나는 K=5로 해서 5개의 문서를 검색하는 것보단  
K=10으로 해서 10개의 문서로 받아온 다음에  
임계값을 0.5로 해서 유사도가 0.5보다 높은 경우만 LLM에 전달해지

In [23]:
print(f"가장 유사한 문서 : {docs3[0][0].page_content}")
print('*'*20)
print(f"가장 유사한 문서의 문서의 유사도 : {docs3[0][1]}")

가장 유사한 문서 : 11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공
********************
가장 유사한 문서의 문서의 유사도 : 0.6830467008459198


## 크로마 문서 추가

In [24]:
for key in db._collection.get():
  print(key)

ids
embeddings
documents
uris
data
metadatas
included


In [25]:
# id를 출력하되 상위 5개만 출력
db._collection.get()['ids'][:5]

['acb714e2-30bc-4d19-88bf-3c5565bde164',
 '804481ad-b60b-40dc-9e5c-041a67b74c32',
 'bc4c602c-c827-4270-9ee5-8eee5fff4b5d',
 'd039f858-6a89-414b-9a1e-d0d0ecd07d59',
 '56b3c1a7-9cdc-49c8-ad92-c83fd6ddcf82']

In [26]:
print('현재 문서 수 :', len(db._collection.get()['ids']))

현재 문서 수 : 69


In [27]:
# 신규 문서 추가 하기
text1 = '''삼성전자가 미국 반도체업체 엔비디아에 고대역폭 메모리(HBM)를 납품하기 위한 테스트를 아직 통과하지 못했다고 로이터통신이 3명의 익명 소식통을 인용해 24일 보도했다.'''
text2 = '''엔비디아발 훈풍으로 전 세계 반도체가 들썩이는 가운데 국내 증시에서는 SK하이닉스 (198,600원 ▼1,400 -0.70%)의 활약이 돋보인다. 엔비디아에 대한 투자자들의 기대치가 높아지고 있어 SK하이닉스 역시 상승세를 이어갈 것이란 전망이 쏟아진다. 다만 SK하이닉스 주가에 상승동력으로 작용했던 엔비디아 실적 재료가 소진된 점은 차익실현 매물이 출현할 가능성을 높인다.'''

랭체인 형식으로 만들어줍니다.

In [28]:
from langchain.docstore.document import Document

new_doc = Document(
    page_content=text2,
    metadata={
        "source": "wareconn.com",
        "page": 1
    }
)

In [29]:
new_doc

Document(metadata={'source': 'wareconn.com', 'page': 1}, page_content='엔비디아발 훈풍으로 전 세계 반도체가 들썩이는 가운데 국내 증시에서는 SK하이닉스 (198,600원 ▼1,400 -0.70%)의 활약이 돋보인다. 엔비디아에 대한 투자자들의 기대치가 높아지고 있어 SK하이닉스 역시 상승세를 이어갈 것이란 전망이 쏟아진다. 다만 SK하이닉스 주가에 상승동력으로 작용했던 엔비디아 실적 재료가 소진된 점은 차익실현 매물이 출현할 가능성을 높인다.')

In [31]:
# 파이썬 리스트
new_docs = [new_doc]

id = db.add_documents(
    new_docs,
    ids=["7"]
)

In [32]:
print('현재 문서 수 :', len(db._collection.get()['ids']))

현재 문서 수 : 70


## 크로마 문서 삭제

`db_fromfile._collection.delete(ids=[doc_id])` 해당 코드로 문서의 id를 찾아내면 문서를 벡터 데이터베이스에서 삭제할 수 있다.

예를 들어 '엔비디아발 훈풍'이라는 텍스트가 포함된 문서를 삭제하고 싶다고 하자. 우선 해당 문서를 검색해서 검색에 잡히는지 확인한다.

In [45]:
# 1. 특정 쿼리로 문서 검색
query = "삼성전자의 사업"
docs = db.similarity_search(query, k=1)  # 가장 유사한 문서 1개 검색

In [46]:
print("검색된 문서:", docs[0].page_content)

검색된 문서: 11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공


벡터 DB에서 모든 id와 document(실제 문서)와 메타 데이터를 추출한다.

Chroma의 get() 메서드는 ids, documents, metadatas 등의 키를 가진 딕셔너리를 반환하는데, 이들 배열은 서로 인덱스가 대응된다. 즉, all_ids[i]는 all_documents[i]와 all_metadatas[i]에 해당하는 문서의 ID다.

In [35]:
# 모든 정보 로드
all_docs = db._collection.get()

# 벡터 DB로부터 id, document, metadata 추출
all_ids = all_docs['ids']
all_documents = all_docs['documents']
all_metadatas = all_docs['metadatas']

In [36]:
all_ids[:2]

['acb714e2-30bc-4d19-88bf-3c5565bde164',
 '804481ad-b60b-40dc-9e5c-041a67b74c32']

In [37]:
all_documents[:2]

['2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등',
 'IM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등\nDS DRAM, NAND Flash, 모바일AP, LCD패널, OLED패널, LED 등 \n∙ 주요 사업은 전자전지기계 등 제조, 전자통신기 등 제도, 컴퓨터 등 제조, 반도체 제조·조립 등. 주요 \n제품과 구성비율은 HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등 54%, TV, 모니터, 냉장고, 세탁기, \n에어컨 등 2 4 % ,  D R A M ,  N A N D  F l a s h ,  모바일 AP 등 19% 등으로 구분 \n∙ 지역별로는 본사를 거점으로 한국 및 CE, IM 부문 산하 해외 9개 지역총괄과 DS 부문 산하 해외 5개 \n지역총괄의 생산ㆍ판매법인 등 165개의 동종업종을 영위하는 종속기업으로 구성\n∙ 국내사업장 : 수원, 구미, 기흥, 화성, 온양, 광주\n∙ 해외사업장 : 86개국 220개지점 (15년 말 기준)\n1) 연결재무재표 기준\n2) 연결재무재표 기준']

In [38]:
all_metadatas[:2]

[{'creationdate': '2017-08-16T00:21:02-08:00',
  'creator': 'nPDF (pdftk 1.41)',
  'moddate': '2017-08-16T00:21:02-08:00',
  'page': 0,
  'page_label': '1',
  'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)',
  'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf',
  'total_pages': 27},
 {'creationdate': '2017-08-16T00:21:02-08:00',
  'creator': 'nPDF (pdftk 1.41)',
  'moddate': '2017-08-16T00:21:02-08:00',
  'page': 0,
  'page_label': '1',
  'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)',
  'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf',
  'total_pages': 27}]

In [39]:
print('id 2개 출력')
print(all_ids[:2])
print('--' * 50)
print('documents 2개 출력')
print(all_documents[:2])
print('--' * 50)
print('metadata 2개 출력')
print(all_metadatas[:2])
print('--' * 50)

id 2개 출력
['acb714e2-30bc-4d19-88bf-3c5565bde164', '804481ad-b60b-40dc-9e5c-041a67b74c32']
----------------------------------------------------------------------------------------------------
documents 2개 출력
['2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등', 'IM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등\nDS DRAM, NAND Flash, 모바일AP, LCD패널, OLED패널, LED 등 \n∙ 주요 사업은 전자전지기계 등 제조, 전자통신기 등 제도, 컴퓨터 등 제조, 반도체 제조·조립 등. 주요 \n제품과 구성비율은 HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등 54%, TV, 모니터, 냉장고, 세탁기, \n에어컨 등 2 4 % ,  D R A M ,  N A N D  F l a s h ,  모바일 AP 등 19% 등으로 구분 \n∙

### 삭제하고자 하는 문서의 ID를 찾아낸다.

In [47]:
# 검색 결과와 일치하는 문서 찾기

# all_documents에 대해서 내가 지우고자 하는 문서인 docs[0]의 내용이랑 일치하는 문서를 일단 찾는다.
# 해당 문서가 몇번째 인지를 찾아내면 그 몇번째인지로부터 id값이 뭔지를 알 수 있다.
for i, doc_content in enumerate(all_documents):
    if docs[0].page_content in doc_content:  # 문서 내용으로 비교
        doc_id = all_ids[i]
print(f"찾은 문서 ID: {doc_id}")

찾은 문서 ID: 3c9d4739-455c-41cb-a54e-4f1bd8134841


In [48]:
# 삭제 전 문서 수
print("삭제 전 문서 수:", db._collection.count())

# 7번 ID의 문서 삭제
db._collection.delete(ids=[doc_id])

# 삭제 후 문서 수
print("삭제 후 문서 수:", db._collection.count())

삭제 전 문서 수: 69
삭제 후 문서 수: 68


## 2. 파이스(Faiss)

In [56]:
!pip install faiss-gpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
from langchain.vectorstores import FAISS

다음은 Langchain에서 FAISS를 이용하여 벡터 DB 사용 시에 가장 간단한 사용법입니다.

```python
# FAISS 를 통해 벡터 저장소 생성. OpenAIEmbeddings()를 사용하거나 HuggingFaceEmbeddings()를 사용하면 된다.
faiss_db = FAISS.from_documents(splited_docs, OpenAIEmbeddings())

# 유사도 검색(쿼리)
similar_docs = faiss_db.similarity_search("사용자의 질문")
```

In [51]:
faiss_db = FAISS.from_documents(splited_docs, model_huggingface)
question = '삼성전자의 주요 사업영역은?'

docs = faiss_db.similarity_search(question)

In [52]:
len(docs)

4

In [58]:
for doc in docs:
  print(doc)
  print('--' * 100)

page_content='11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공' metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 9, 'page_label': '10'}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------

You can also save and load a FAISS index. This is useful so you don't have to recreate it everytime you use it.



In [59]:
# 파일 저장
faiss_db.save_local('faiss_index')

## 파일 불러오기
new_db_faiss = FAISS.load_local('faiss_index', model_huggingface, allow_dangerous_deserialization=True)
question = '삼성전자의 주요 사업영역은?'

docs = new_db_faiss.similarity_search(question)

In [60]:
for doc in docs:
  print(doc)
  print('--' * 100)

page_content='11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공' metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 9, 'page_label': '10'}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [61]:
print('저장된 문서 개수:', faiss_db.index.ntotal)

저장된 문서 개수: 69


In [62]:
# 저장된 문서들 출력
faiss_db.docstore._dict

{'dcc6362d-7f35-4752-ab8d-05c39db4a7f2': Document(id='dcc6362d-7f35-4752-ab8d-05c39db4a7f2', metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}, page_content='2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등'),
 'a3e7f7a6-5e58-4b29-b959-6ec62afd029d': Document(id='a3e7f7a

## 파이스 문서 추가

### 문서 1개 추가

In [63]:
from langchain.schema import Document

In [64]:
new_doc = Document(
    page_content="엔비디아발 훈풍으로 전 세계 반도체가 들썩이는 가운데 국내 증시에서는 SK하이닉스 (198,600원 ▼1,400 -0.70%)의 활약이 돋보인다. 엔비디아에 대한 투자자들의 기대치가 높아지고 있어 SK하이닉스 역시 상승세를 이어갈 것이란 전망이 쏟아진다. 다만 SK하이닉스 주가에 상승동력으로 작용했던 엔비디아 실적 재료가 소진된 점은 차익실현 매물이 출현할 가능성을 높인다.",
    metadata={"source": "내부 문서", "author": "홍길동"}
)

In [65]:
# 기존 문서
print("count before:", faiss_db.index.ntotal)

count before: 69


In [66]:
faiss_db.add_documents([new_doc])

['fd98e638-4473-466d-8dea-ddb4358abf72']

In [67]:
print("count after:", faiss_db.index.ntotal)

count after: 70


### 다수의 문서 추가

In [68]:
# 2. 여러 문서 추가
new_docs = [
    Document(page_content="첫 번째 새 문서", metadata={"source": "파일1"}),
    Document(page_content="두 번째 새 문서", metadata={"source": "파일2"}),
    Document(page_content="세 번째 새 문서", metadata={"source": "파일3"})
]

# 여러 문서 추가
faiss_db.add_documents(new_docs)
print("count after:", faiss_db.index.ntotal)

count after: 73


### 병합 방법으로 추가

In [69]:
# 신규 문서 추가 하기
text1 = '''삼성전자가 미국 반도체업체 엔비디아에 고대역폭 메모리(HBM)를 납품하기 위한 테스트를 아직 통과하지 못했다고 로이터통신이 3명의 익명 소식통을 인용해 24일 보도했다.'''
text2 = '''엔비디아발 훈풍으로 전 세계 반도체가 들썩이는 가운데 국내 증시에서는 SK하이닉스 (198,600원 ▼1,400 -0.70%)의 활약이 돋보인다. 엔비디아에 대한 투자자들의 기대치가 높아지고 있어 SK하이닉스 역시 상승세를 이어갈 것이란 전망이 쏟아진다. 다만 SK하이닉스 주가에 상승동력으로 작용했던 엔비디아 실적 재료가 소진된 점은 차익실현 매물이 출현할 가능성을 높인다.'''

In [70]:
new_db = FAISS.from_texts([text1, text2], model_huggingface)

In [71]:
print('신규 DB 샘플 개수:', new_db.index.ntotal)

신규 DB 샘플 개수: 2


In [72]:
# 저장된 문서들 출력
new_db.docstore._dict

{'41a463d9-f03b-4f88-beee-8e51253f11d6': Document(id='41a463d9-f03b-4f88-beee-8e51253f11d6', metadata={}, page_content='삼성전자가 미국 반도체업체 엔비디아에 고대역폭 메모리(HBM)를 납품하기 위한 테스트를 아직 통과하지 못했다고 로이터통신이 3명의 익명 소식통을 인용해 24일 보도했다.'),
 'a21dbc55-699d-43fe-8970-3a224d52c3b5': Document(id='a21dbc55-699d-43fe-8970-3a224d52c3b5', metadata={}, page_content='엔비디아발 훈풍으로 전 세계 반도체가 들썩이는 가운데 국내 증시에서는 SK하이닉스 (198,600원 ▼1,400 -0.70%)의 활약이 돋보인다. 엔비디아에 대한 투자자들의 기대치가 높아지고 있어 SK하이닉스 역시 상승세를 이어갈 것이란 전망이 쏟아진다. 다만 SK하이닉스 주가에 상승동력으로 작용했던 엔비디아 실적 재료가 소진된 점은 차익실현 매물이 출현할 가능성을 높인다.')}

In [73]:
faiss_db.merge_from(new_db)

In [74]:
print("count after merge:", faiss_db.index.ntotal)

count after merge: 75


## 문서 삭제

In [75]:
# 1. 특정 쿼리로 문서 검색
query = "삼성전자가 미국 반도체업체 엔비디아"
docs = faiss_db.similarity_search(query, k=1)
print(docs)

[Document(id='41a463d9-f03b-4f88-beee-8e51253f11d6', metadata={}, page_content='삼성전자가 미국 반도체업체 엔비디아에 고대역폭 메모리(HBM)를 납품하기 위한 테스트를 아직 통과하지 못했다고 로이터통신이 3명의 익명 소식통을 인용해 24일 보도했다.')]


In [76]:
# 2. 문서 ID 확인
doc = docs[0]
doc_id = doc.id
print(doc_id)

41a463d9-f03b-4f88-beee-8e51253f11d6


In [77]:
# 3. 문서 삭제
faiss_db.delete([doc_id])
print("count after:", faiss_db.index.ntotal)

count after: 74


## 3. BM25

BM25는 TF-IDF를 개선한 텍스트 검색 알고리즘으로, 문서의 관련성을 계산하는 데 효과적입니다.

In [78]:
!pip install rank_bm25


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [79]:
from langchain.retrievers import BM25Retriever

In [80]:
# BM25 검색기 초기화
bm25_retriever = BM25Retriever.from_documents(splited_docs)

In [81]:
# 키워드 기반 검색
docs = bm25_retriever.get_relevant_documents("삼성전자의 사업 영역은")

for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 10, 'page_label': '11'}
:
12 Q2 삼성전자의 Harman사 인수에 대해 어떻게 생각하십니까 ? A  삼성전자가 최근 80억달러(약 9조)에 미국의 전장업체(자동차 전자기기에 대한 사업)인 Harman 사를  인수하기로 결정했습니다. 하만사는 세계적으로 자동차용 인포테인먼트와 텔레매틱스 시장에서 점유 율 1~2위를 달리고 있으며 매출도 상당합니다. 이번 삼성전자의 하만 인수는 모바일 및 가전 시장에서  나아가 Connected Car 시장 진입에 진출했다는 것에 큰 의의가 있습니다. 향후 10년 시장 내 Connected  Car의 비중은 90%에 육박할 전망입니다. LG전자 역시 자동차부품(VC) 사업 부문에 있어 상당한 경쟁  위협 요인가 될 것으로 내다봤습니다. <관련기사> 전장사업 강자 하만과 손잡은 삼성…스마트카 ‘티어1’ 노린다 (한국경제. 2016-11-21) 삼성전자는 자동차 전장(電裝)사업에서 후발 주자다. 2005년 전장사업을 시작한 LG전자는
474
******************************
{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:0

C:\Users\user\AppData\Local\Temp\ipykernel_14864\2483167931.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = bm25_retriever.get_relevant_documents("삼성전자의 사업 영역은")
